### Recuperation d'information ciblée

Nous avons vu comment parser une page, récupérer sans distinction les informations d'une page web

Mais, en général, le but de scrapper est d'automatiser la collecte d'informations ciblées sur le web



In [3]:
from bs4 import BeautifulSoup
import requests


Imaginons que je veuille scraper la description des derniers films sortie en salle

Je vais donc sur le site d'allo ciné et j'essaie de trouver les balises qui me donneront les liens vers les pages spécifiques de ces films pour ensuit y recuperer leurs résumés

#### Récupération des url des pages de films fraichement sorties en salle

In [4]:
url='http://www.allocine.fr/'
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content,'html')
soup

http://www.allocine.fr/ 200


<!DOCTYPE html>
<html>
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow" name="robots"/>
<title>AlloCiné : Cinéma, Séries TV, BO de films et séries, Vidéos, DVD et VOD</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="AlloCiné, le site de référence du cinéma et des séries tv ! Découvrez notre recherche d'horaires de films, le programme tv de vos séries préférées, l'actualité ciné et séries, les émissions AlloCiné, les dernières 

Sur mon navigateur web (Chrome, mozilla ...) j'utilise la fonction "inspecter" (click droit / inspecter) et je fais glisser ma souris vers les zones de la page qui m'interesse. Je verrai en meme temps le script html se deplacer vers les instructions du script html en question. 

C'est comme ça que je fais pour reperer les balises qui m'interessent

Ainsi, je remarque que le lien relatif de la page web specifique au film nouveau à l'affiche est rangé dans ces balises :

a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a


In [6]:
for p in soup.find_all('a'):
    print (p.text)

Bohemian Rhapsody
Mowgli
Game of Thrones
The Walking Dead
séances cinéma
news
dossiers
émissions AlloCiné
dernières bandes-annonces

            Cinéma
        

            Séries
        
 B.O.
        

            DVD
        

            VOD
        
 Netflix
        
Avant-premières
Agenda des sorties
Films pour enfants à l'affiche
Meilleurs films
Box Office
Meilleurs films 2017
Bandes-annonces de films
BA des films prochainement au cinéma
Tous les films pour enfants
Tous les films
Sorties de la semaine
Séances
Bande-annonce
Le Grand Bain
Séances
Bande-annonce
Chair de poule 2 : Les Fantômes d'Halloween
Séances
Bande-annonce
Halloween
Séances
Bande-annonce
Jean-Christophe & Winnie
Séances
Bande-annonce
Cold War
Séances
Bande-annonce
Quién te cantará
Séances
Bande-annonce
La Tendre indifférence du monde
Séances
Bande-annonce
Bamse au pays des voleurs
Séances
La Fille du Far-West (Met - Pathé Live)
Séances
Die Walküre (Royal Opera House)
Dilili à Paris
Alad'2
Yéti & Compagnie
Veno

Cette fois ci le site est plus difficile à "extraire", utilisons des parametres supplementaires à cette fonction de recherche, find_all, bien plus spécifique que celle actuellement utilisée

In [12]:
# En plus de la balise a, bien identifiée, nous remarquons quelques informations supplementaires comme 
# la valeur de la variable class de ces balises identiques
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    print(elem)

<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=247007.html" title="Chair de poule 2 : Les Fantômes d'Halloween">Chair de poule 2 : Les Fantômes d'Halloween</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=238622.html" title="Halloween">Halloween</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=251837.html" title="Jean-Christophe &amp; Winnie">Jean-Christophe &amp; Winnie</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=252203.html" title="Cold War">Cold War</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=253056.html" title="Quién te cantará">Quién te cantará</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=264018.html" title="La Tendre indifférence du monde">La Tendre indifférence du monde</a>
<a class="meta-title meta

#### Recuperation des href

Nous avons remarqué la présences des liens href vers les pages qui nous interressent. Allons les récuperer

In [16]:
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    # Tout simplement
    print(elem.get('href'))
    # Je range tout cela dans une liste 

Le Grand Bain
Chair de poule 2 : Les Fantômes d'Halloween
Halloween
Jean-Christophe & Winnie
Cold War
Quién te cantará
La Tendre indifférence du monde
Bamse au pays des voleurs
La Fille du Far-West (Met - Pathé Live)
Die Walküre (Royal Opera House)
Dilili à Paris
Alad'2
Yéti & Compagnie
Venom
Le Jeu
Le Flic de Belleville
A Star Is Born
La Prophétie de l'horloge
Élite
Le Bureau des Légendes
The Haunting of Hill House
La Casa De Papel
Tell Me a Story
Homecoming
Nightflyers
Ad Vitam


Pouvez vous me récupérer les titres via la recherche de 'title' dans les éléments de la liste précedente ?

#### Recupération des resumés

Commençon par construire les url que nous allons utiliser pour y recupérer les résumés

Commencez par ranger les valeurs de href dans une liste liens


In [24]:
liens=[]
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    # Tout simplement
    liens.append(elem.get('href'))
    # Je range tout cela dans une liste 
liens

['/film/fichefilm_gen_cfilm=235582.html',
 '/film/fichefilm_gen_cfilm=247007.html',
 '/film/fichefilm_gen_cfilm=238622.html',
 '/film/fichefilm_gen_cfilm=251837.html',
 '/film/fichefilm_gen_cfilm=252203.html',
 '/film/fichefilm_gen_cfilm=253056.html',
 '/film/fichefilm_gen_cfilm=264018.html',
 '/film/fichefilm_gen_cfilm=232881.html',
 '/film/fichefilm_gen_cfilm=263344.html',
 '/film/fichefilm_gen_cfilm=263607.html',
 '/film/fichefilm_gen_cfilm=243835.html',
 '/film/fichefilm_gen_cfilm=242449.html',
 '/film/fichefilm_gen_cfilm=230803.html',
 '/film/fichefilm_gen_cfilm=138473.html',
 '/film/fichefilm_gen_cfilm=259649.html',
 '/film/fichefilm_gen_cfilm=257706.html',
 '/film/fichefilm_gen_cfilm=173109.html',
 '/film/fichefilm_gen_cfilm=257034.html',
 '/series/ficheserie_gen_cserie=22373.html',
 '/series/ficheserie_gen_cserie=17907.html',
 '/series/ficheserie_gen_cserie=21978.html',
 '/series/ficheserie_gen_cserie=21504.html',
 '/series/ficheserie_gen_cserie=23036.html',
 '/series/ficheseri

L url absolue des pages de films recherchés se construit dns cette forme : http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html

Il faut donc reprendre la liste precedente et construire les urls absolues pour not recherche

A vous de jouer

nb: Ne prenez pas les liens pour les series

In [38]:
liens_film=['http://www.allocine.fr'+elem for elem in liens if 'film' in elem]
liens_film

['http://www.allocine.fr/film/fichefilm_gen_cfilm=235582.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=247007.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=238622.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=251837.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=252203.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=253056.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=264018.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=232881.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=263344.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=263607.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=242449.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=230803.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=138473.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=259649.html',
 'http://www.allocine.fr/film/fichefilm_

Enfin, sur chacune des pages, il faut récupérer le titre et le synopsis. Essayons pour un film, le premier de la liste de lien

In [27]:
url=liens_film[0]
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content,'html')
soup

http://www.allocine.fr/film/fichefilm_gen_cfilm=235582.html 200


<!DOCTYPE html>
<html>
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow" name="robots"/>
<title>Le Grand Bain - film 2018 - AlloCiné</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="Le Grand Bain est un film réalisé par Gilles Lellouche avec Mathieu Amalric, Guillaume Canet. Synopsis : C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité tout

Pour le titre

div class="titlebar-title titlebar-title-lg">Le Grand Bain</div

Pour le synopsis

div class="content-txt " itemprop="description"
    
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
    
      </div

In [30]:
for elem in soup.find_all('div',attrs={"class" :"titlebar-title titlebar-title-lg"}):
    # Tout simplement
    print(elem.text)
    
for elem in soup.find_all('div',attrs={"class" :"content-txt "}):
    # Tout simplement
    print(elem.text)

Le Grand Bain

    
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
    
      


1) Automatisez ce script pour toute la liste

2) Rangez les informations dans trois listes (liens_film, titre, synopsis)

3) Créer un dataframe qui reprend ces trois informations dans trois colonnes associées

4) Sauvegardez ce dataframe dans un fichier csv

Et voilà votre premier vrai scrap, vous êtes de vrais hackers maintenant

In [39]:
import time
import random
from random import randint

titre=[]
synopsis=[]

for lien in liens_film:
    
    url=lien
    # Je ralentit la fréquence des requetes pour ne pas être identifié et donc bannir du site
    time.sleep(random.uniform(1.0, 2.0))
    r = requests.get(url)
    print(url, r.status_code)
    soup = BeautifulSoup(r.content,'html')
    
    
    for elem in soup.find_all('div',attrs={"class" :"titlebar-title titlebar-title-lg"}):
        titre.append(elem.text.strip())

    for elem in soup.find_all('div',attrs={"class" :"content-txt "}):
        # Tout simplement
        synopsis.append(elem.text.strip())
        
# Je verifie la longueur des listes avant la creation du df
len(titre),len(synopsis),len(liens_film)

http://www.allocine.fr/film/fichefilm_gen_cfilm=235582.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=247007.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=238622.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=251837.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=252203.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=253056.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=264018.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=232881.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=263344.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=263607.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=242449.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=230803.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=138473.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=259649.html 200
http://www.allocine.fr/film/fichefilm_ge

(18, 18, 18)

In [40]:
import pandas as pd
df=pd.DataFrame({'Titre':titre})
df['synopsis']=synopsis
df['liens']=liens_film

In [41]:
df.to_csv('data/allo_cine.csv', index=False)